# 1.准备环境与路径

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

In [5]:
# def CreateSparkContext():
#     '''
#     spark配置：
#     1.显示在spark或 hadoop-yarn UI界面的App名称
#     2.设置不显示spark执行进度以免界面太乱
#     '''
#     sparkConf = SparkConf().setAppName('Finacial Distress Prediction')  \
#             .set('spark.ui.showConsoleProgress','false') 
#     sc = SparkContext(conf=sparkConf)
#     print('master='+sc.master)
#     SetLogger(sc) #设置不要显示太多信息
#     SetPath(sc) # 设置文件读取路径
#     return sc

# def SetLogger(sc):
#     logger = sc._jvm.org.apache.log4j
#     logger.LogManager.getLogger("org").setLevel(logger.Level.ERROR)
#     logger.LogManager.getLogger("akka").setLevel(logger.Level.ERROR)
#     logger.LogManager.getRootLogger().setLevel(logger.Level.ERROR)
    
# def SetPath(sc):
#     if sc.master[0:5]=="local":
#         Path = "file:/home/hadoop/eclipse-workspace/FinacialDistress/"
#     else:
#         Path = "hdfs://ubuntu:9000/sparkproject/FinacialDistress/"
#     return Path

In [60]:
spark = SparkSession.builder.master("local").getOrCreate()
# Path = SetPath(sc)
Path = "file:/home/hadoop/eclipse-workspace/FinancialDistressPre/"

# 2.读取数据并整理

In [13]:
raw_df = spark.read.format("csv") \
                .option("header","true") \
                .option("delimiter",",") \
                .load(Path+"data/FinacialDistress.csv")



raw_df.count()

type(raw_df)

raw_df.printSchema()

raw_df.select('Company','Time','Financial Distress','x1','x2').show()



from pyspark.sql.functions import udf,col
import pyspark.sql.types
def replace_question(col): # df.na.fill() 或者df.fillna()
    return ('0' if col=="?" else col)
replace_question = udf(replace_question)

rawDF = raw_df.select(['Company']+[replace_question(col("Time")).cast('int').alias('Time')] +
                       [replace_question(col(c)).cast('double').alias(c) for c in raw_df.columns[2:]])


df = rawDF.withColumn('Financial Distress01',(rawDF['Financial Distress']>-0.5).cast('int'))

train_df,test_df = df.randomSplit([0.7,0.3])
# train_df.cache()
# test_df.cache()

root
 |-- Company: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Financial Distress: string (nullable = true)
 |-- x1: string (nullable = true)
 |-- x2: string (nullable = true)
 |-- x3: string (nullable = true)
 |-- x4: string (nullable = true)
 |-- x5: string (nullable = true)
 |-- x6: string (nullable = true)
 |-- x7: string (nullable = true)
 |-- x8: string (nullable = true)
 |-- x9: string (nullable = true)
 |-- x10: string (nullable = true)
 |-- x11: string (nullable = true)
 |-- x12: string (nullable = true)
 |-- x13: string (nullable = true)
 |-- x14: string (nullable = true)
 |-- x15: string (nullable = true)
 |-- x16: string (nullable = true)
 |-- x17: string (nullable = true)
 |-- x18: string (nullable = true)
 |-- x19: string (nullable = true)
 |-- x20: string (nullable = true)
 |-- x21: string (nullable = true)
 |-- x22: string (nullable = true)
 |-- x23: string (nullable = true)
 |-- x24: string (nullable = true)
 |-- x25: string (nullable = true)
 |--

In [282]:
train_df.count()/df.count()

0.6996187363834423

In [4]:
from pyspark.sql.functions import col

# 3.处理字符串分类特征
* 利用StringIndexer将文字分类特征转换为数值分类特征
* 利用OneHotEncoder将数值分类特征转换为Vector如(0,0,0,1,0,1,0,0)

In [ ]:
# from pyspark.ml.feature import StringIndexer,OneHotEncoder
# # -----StringIndexer----
# category = StringIndexer(inputCol='inputStr',outputCOl='outputNum') #StringIndexer是一个Estimator
# categoryTransformer = category.fit(df) #调用fit方法后生成一个Transformer
# categoryTransformer.labels #即为分类字符串
# #将Transformer应用到train_df和test_df
# trainDF = categoryTransformer.transform(train_df)
# testDF = categoryTransformer.transform(test_df)

# # -------OneHotEncoder------
# encoder = OneHotEncoder(inputCol='outputNum',outputCol='outputVec',dropLast=False)
# train_DF = encoder.transform(trainDF)
# testDF = encoder.transform(testDF)

# 4.使用 VectorAssembler 将所有特征集成为一个特征Vector

In [14]:
from pyspark.ml.feature import VectorAssembler
assemblers = df.columns[3:-1]
vecAssembler = VectorAssembler(inputCols = assemblers,outputCol= 'Features')
trainDF = vecAssembler.transform(train_df)
# testDF = vecAssembler.transform(test_df)

In [285]:
trainDF.select('Features').show()
trainDF.select('Features').take(1)

+--------------------+
|            Features|
+--------------------+
|[1.281,0.022934,0...|
|[1.1131,-0.015229...|
|[1.4809,0.11739,0...|
|[1.3854,0.1013,0....|
|[1.267,0.053775,0...|
|[1.2675,0.059595,...|
|[1.3881,0.10079,0...|
|[1.4323,0.083595,...|
|[1.3399,0.066151,...|
|[1.3167,0.061076,...|
|[1.3457,0.049124,...|
|[1.3107,0.028949,...|
|[1.3833,0.087312,...|
|[1.3833,0.013043,...|
|[1.0246,0.13107,0...|
|[1.1007,0.059918,...|
|[1.3134,0.065683,...|
|[0.98085,0.014711...|
|[0.88577,0.051745...|
|[0.94864,0.040847...|
+--------------------+
only showing top 20 rows



[Row(Features=DenseVector([1.281, 0.0229, 0.8745, 1.2164, 0.0609, 0.1883, 0.5251, 0.0189, 0.1828, 0.0064, 0.8582, 2.0058, 0.1255, 6.9706, 4.6512, 0.0501, 2.1984, 0.0183, 0.025, 0.0273, 1.4173, 9.5554, 0.1487, 0.67, 214.76, 12.641, 6.4607, 0.0438, 0.2046, 0.3518, 8.3161, 0.2892, 0.7661, 2.5825, 77.4, 0.0267, 1.6307, 0.015, 0.0055, 0.1273, 9.6951, -0.7362, 0.9856, 0.1802, 1.5006, 0.0262, 7.0513, 1174.9, 5.3399, 0.8513, 12.837, 0.0617, 0.1809, 209.87, -0.5826, 0.471, 0.1099, 0.0, 0.0, 0.2201, 7.1241, 15.381, 3.2702, 17.872, 34.692, 30.087, 12.8, 7991.4, 364.95, 15.8, 61.476, 4.0, 36.0, 85.437, 27.07, 26.102, 16.0, 16.0, 0.2, 22.0, 0.0604, 30.0, 49.0]))]

# 5.使用DecisionTreeClassifier二元分类

In [286]:
from pyspark.ml.classification import DecisionTreeClassifier
dtClassifier = DecisionTreeClassifier(labelCol='Financial Distress01',featuresCol='Features',impurity='gini',maxDepth=10,maxBins=14)
dtModel = dtClassifier.fit(trainDF)

In [287]:
dtPredict = dtModel.transform(testDF) #预测

In [288]:
dtPredict.columns

DataFrame[Company: string, Time: int, Financial Distress: double, x1: double, x2: double, x3: double, x4: double, x5: double, x6: double, x7: double, x8: double, x9: double, x10: double, x11: double, x12: double, x13: double, x14: double, x15: double, x16: double, x17: double, x18: double, x19: double, x20: double, x21: double, x22: double, x23: double, x24: double, x25: double, x26: double, x27: double, x28: double, x29: double, x30: double, x31: double, x32: double, x33: double, x34: double, x35: double, x36: double, x37: double, x38: double, x39: double, x40: double, x41: double, x42: double, x43: double, x44: double, x45: double, x46: double, x47: double, x48: double, x49: double, x50: double, x51: double, x52: double, x53: double, x54: double, x55: double, x56: double, x57: double, x58: double, x59: double, x60: double, x61: double, x62: double, x63: double, x64: double, x65: double, x66: double, x67: double, x68: double, x69: double, x70: double, x71: double, x72: double, x73: do

# 6.建立机器学习流程pipeline

In [17]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vecAssembler,dtClassifier])
pipeline.getStages()

In [290]:
pipelineModel = pipeline.fit(train_df)

In [291]:
pipelineModel.stages[1]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 10 with 145 nodes

In [292]:
dtModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 10 with 145 nodes

In [293]:
print(pipelineModel.stages[1].toDebugString[:500]) #查看决策树规则

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 10 with 145 nodes
  If (feature 45 <= 0.0022849999999999997)
   If (feature 2 <= 0.734785)
    If (feature 22 <= 0.17000500000000002)
     If (feature 61 <= 12.175)
      If (feature 19 <= 0.25765499999999997)
       If (feature 60 <= 6.92695)
        If (feature 68 <= -46.2455)
         If (feature 16 <= 1.3288)
          Predict: 0.0
         Else (feature 16 > 1.3288)
          Predict: 1.0
        Else


In [294]:
pipelinePredict = pipelineModel.transform(test_df) #预测

In [295]:
pipelinePredict.select('Company','Time','Features','Financial Distress01','rawPrediction','probability','prediction').show()

+-------+----+--------------------+--------------------+-------------+--------------------+----------+
|Company|Time|            Features|Financial Distress01|rawPrediction|         probability|prediction|
+-------+----+--------------------+--------------------+-------------+--------------------+----------+
|      1|   2|[1.27,0.0064542,0...|                   1|    [0.0,4.0]|           [0.0,1.0]|       1.0|
|      1|   3|[1.0529,-0.059379...|                   1|   [15.0,0.0]|           [1.0,0.0]|       0.0|
|     10|  12|[1.5795,0.17996,0...|                   1| [0.0,1436.0]|           [0.0,1.0]|       1.0|
|     10|  13|[1.5012,0.078785,...|                   1|  [0.0,323.0]|           [0.0,1.0]|       1.0|
|    100|   2|[0.79696,0.090335...|                   1| [0.0,1436.0]|           [0.0,1.0]|       1.0|
|    100|   6|[0.9303,0.084578,...|                   1| [0.0,1436.0]|           [0.0,1.0]|       1.0|
|    100|  11|[1.1898,0.10814,0...|                   1| [0.0,1436.0]|   

# 7.评估模型准确率

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol='Financial Distress01',
                                          metricName='areaUnderROC')
auc = evaluator.evaluate(pipelinePredict)

In [297]:
auc

0.9042644896303433

# 8.寻找最佳模型

## 8.1使用TrainValidationSplit寻找最佳模型

In [22]:
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit
paramGrid = ParamGridBuilder().addGrid(dtClassifier.impurity,["gini","entropy"]) \
                .addGrid(dtClassifier.maxDepth,[5,10,15,20]) \
                .addGrid(dtClassifier.maxBins,[10,15,20,25]).build()
dtTVS = TrainValidationSplit(estimator = dtClassifier,evaluator=evaluator,
                               estimatorParamMaps = paramGrid,trainRatio=0.7)
dtTVS_pipeline = Pipeline(stages=[vecAssembler,dtTVS])

In [299]:
dtTVS_models = dtTVS_pipeline.fit(train_df)

In [300]:
bestModel = dtTVS_models.stages[1].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 5 with 53 nodes

In [301]:
print(bestModel.toDebugString[:500])

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 5 with 53 nodes
  If (feature 8 <= -0.114315)
   If (feature 57 <= -9.549999999999999E-5)
    If (feature 10 <= 0.50169)
     If (feature 64 <= 34.1225)
      Predict: 1.0
     Else (feature 64 > 34.1225)
      If (feature 0 <= 0.7623949999999999)
       Predict: 0.0
      Else (feature 0 > 0.7623949999999999)
       Predict: 1.0
    Else (feature 10 > 0.50169)
     If (feature 81 <= 12.5)
      Predict: 0


In [302]:
dtTVS_predict = dtTVS_models.transform(test_df)
auc = evaluator.evaluate(dtTVS_predict)
auc

0.9033390801683483

## 8.2使用crossValidation寻找最佳模型

In [303]:
from pyspark.ml.tuning import CrossValidator
dtCV = CrossValidator(estimator=dtClassifier,evaluator=evaluator,
                    estimatorParamMaps = paramGrid,numFolds=3)
dtCV_pipeline = Pipeline(stages=[vecAssembler,dtCV])

In [304]:
dtCV_models = dtCV_pipeline.fit(train_df)

In [305]:
bestModel_ = dtCV_models.stages[1].bestModel
bestModel_

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4c8989b34e2c2d43789c) of depth 5 with 55 nodes

In [306]:
dtCV_predict = dtCV_models.transform(test_df)
auc = evaluator.evaluate(dtCV_predict)
auc

0.9061279853962781

# 9.随机森林

In [20]:
from pyspark.ml.classification import RandomForestClassifier
rfClassifier = RandomForestClassifier(labelCol = 'Financial Distress01',featuresCol='Features',numTrees = 10)
rf_pipeline = Pipeline(stages=[vecAssembler,rfClassifier])
rf_models = rf_pipeline.fit(train_df)
rfPredict = rf_models.transform(test_df)
auc = evaluator.evaluate(rfPredict)
auc

0.9241099881635253

## 9.1 使用TrainValidationSplit寻找最佳模型

In [24]:
paramGrid = ParamGridBuilder().addGrid(rfClassifier.impurity,["gini","entropy"]) \
                .addGrid(rfClassifier.maxDepth,[5,10,15]) \
                .addGrid(rfClassifier.maxBins,[10,15,20]) \
                .addGrid(rfClassifier.numTrees,[10,20]).build()
rfTVS = TrainValidationSplit(estimator = rfClassifier,evaluator=evaluator,
                               estimatorParamMaps = paramGrid,trainRatio=0.7)
rfTVS_pipeline  = Pipeline(stages=[vecAssembler,rfTVS])
rfTVS_models = rfTVS_pipeline.fit(train_df)
rfTVS_predict = rfTVS_models.transform(test_df)
auc = evaluator.evaluate(rfTVS_predict)
auc

0.9373918783574616

In [42]:
print(rfTVS_models.stages[1].bestModel.totalNumNodes)
print(rfTVS_models.stages[1].bestModel.treeWeights)
print(rfTVS_models.stages[1].bestModel.getNumTrees)
#  'numClasses',
#  'numFeatures',
#  'numTrees',
#  'params',
#  'thresholds',
#  'toDebugString',
#  'totalNumNodes',
#  'treeWeights',
#  'trees',

818
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
20


## 9.2使用crossValidation寻找最佳模型

In [309]:
rfCV = CrossValidator(estimator=rfClassifier,evaluator=evaluator,
                    estimatorParamMaps = paramGrid,numFolds=3)

rfCV_pipeline = Pipeline(stages=[vecAssembler,rfCV])
rfCV_models = rfCV_pipeline.fit(train_df)

In [310]:
rfCV_predict = rfCV_models.transform(test_df)
auc = evaluator.evaluate(rfCV_predict)
auc

0.9351706302925814

# 10 决策树回归

In [43]:
from pyspark.ml.regression import DecisionTreeRegressor
dtReg = DecisionTreeRegressor(labelCol='Financial Distress',featuresCol='Features')
dtReg_pipeline = Pipeline(stages=[vecAssembler,dtReg])
dtReg_model = dtReg_pipeline.fit(train_df)
dtReg_predict = dtReg_model.transform(test_df)

In [314]:
dtReg_predict.select('Company','Time','Features','Financial Distress','prediction').show()

+-------+----+--------------------+------------------+--------------------+
|Company|Time|            Features|Financial Distress|          prediction|
+-------+----+--------------------+------------------+--------------------+
|      1|   2|[1.27,0.0064542,0...|          -0.45597|-0.42818420844155847|
|      1|   3|[1.0529,-0.059379...|          -0.32539|-0.42818420844155847|
|     10|  12|[1.5795,0.17996,0...|            1.1553|  0.8798071445645164|
|     10|  13|[1.5012,0.078785,...|           0.31012|  0.8798071445645164|
|    100|   2|[0.79696,0.090335...|           0.13046| 0.22446443815899586|
|    100|   6|[0.9303,0.084578,...|           0.47346| 0.22446443815899586|
|    100|  11|[1.1898,0.10814,0...|           0.04456|  0.4636442058004643|
|    100|  12|[1.5872,0.0024181...|           -0.1284|0.012763109456521747|
|    100|  13|[1.3346,-0.1506,0...|          -0.48243|-0.42818420844155847|
|    101|   2|[1.1733,0.11988,0...|            1.2129|   1.411190434322034|
|    101|   

In [44]:
# 模型评估
from pyspark.ml.evaluation import RegressionEvaluator
reg_evaluator = RegressionEvaluator(labelCol='Financial Distress',predictionCol = 'prediction',metricName='rmse')
rmse = reg_evaluator.evaluate(dtReg_predict)
rmse

In [340]:
RegressionEvaluator(labelCol='Financial Distress',predictionCol = 'prediction',metricName='r2').evaluate(dtReg_predict)

0.44734030206570463

In [46]:
# TrainValidation
paramGrid = ParamGridBuilder().addGrid(dtReg.maxDepth,[5,10,15]) \
                              .addGrid(dtReg.maxBins,[10,15,20]).build()

dtRegTVS = TrainValidationSplit(estimator = dtReg,evaluator=reg_evaluator,
                               estimatorParamMaps = paramGrid,trainRatio=0.7)
dtRegTVS_pipeline  = Pipeline(stages=[vecAssembler,dtRegTVS])
dtRegTVS_models = dtRegTVS_pipeline.fit(train_df)
dtRegTVS_predict = dtRegTVS_models.transform(test_df)
rmse = reg_evaluator.evaluate(dtRegTVS_predict)
rmse

1.4569681896173268

In [49]:
dtRegTVS_models.stages[1].bestModel.depth
dtRegTVS_models.stages[1].bestModel.numNodes

53

In [323]:
# crossValidation
dtRegCV = CrossValidator(estimator=dtReg,evaluator=reg_evaluator,
                    estimatorParamMaps = paramGrid,numFolds=3)

dtRegCV_pipeline = Pipeline(stages=[vecAssembler,dtRegCV])
dtRegCV_models = dtRegCV_pipeline.fit(train_df)
dtRegCV_predict = dtRegCV_models.transform(test_df)
rmse = reg_evaluator.evaluate(dtRegCV_predict)

In [324]:
rmse

1.0442384266167395

# 11 GBT Regression

In [50]:
from pyspark.ml.regression import GBTRegressor
gbtReg = GBTRegressor(labelCol='Financial Distress',featuresCol='Features')
gbtReg_pipeline = Pipeline(stages=[vecAssembler,gbtReg])
gbtReg_models = gbtReg_pipeline.fit(train_df)
gbtReg_predict = gbtReg_models.transform(test_df)
rmse = reg_evaluator.evaluate(gbtReg_predict)
rmse

In [51]:
# TrainValidationSplit
paramGrid = ParamGridBuilder().addGrid(gbtReg.maxDepth,[5,10]) \
                              .addGrid(gbtReg.maxBins,[10,15])  \
                              .addGrid(gbtReg.maxIter,[10,30]).build()

gbtRegTVS = TrainValidationSplit(estimator = gbtReg,evaluator=reg_evaluator,
                               estimatorParamMaps = paramGrid,trainRatio=0.7)
gbtRegTVS_pipeline  = Pipeline(stages=[vecAssembler,gbtRegTVS])
gbtRegTVS_models = gbtRegTVS_pipeline.fit(train_df)
gbtRegTVS_predict = gbtRegTVS_models.transform(test_df)
rmse = reg_evaluator.evaluate(gbtRegTVS_predict)
rmse

1.2146728441588632

In [59]:
gbtRegTVS_models.stages[1].bestModel.totalNumNodes
gbtRegTVS_models.stages[1].bestModel.getNumTrees

10

In [ ]:
# crossValidation
gbtRegCV = CrossValidator(estimator=gbtReg,evaluator=reg_evaluator,
                    estimatorParamMaps = paramGrid,numFolds=3)

gbtRegCV_pipeline = Pipeline(stages=[vecAssembler,gbtRegCV])
gbtRegCV_models = gbtRegCV_pipeline.fit(train_df)
gbtRegCV_predict = gbtRegCV_models.transform(test_df)
rmse = reg_evaluator.evaluate(gbtRegCV_predict)
rmse

In [2]:
import pyspark.sql.functions

In [3]:
dir(pyspark.sql.functions)

['AutoBatchedSerializer',
 'Column',
 'DataFrame',
 'DataType',
 'PandasUDFType',
 'PickleSerializer',
 'PythonEvalType',
 'SparkContext',
 'StringType',
 'UserDefinedFunction',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_binary_mathfunctions',
 '_collect_list_doc',
 '_collect_set_doc',
 '_create_binary_mathfunction',
 '_create_function',
 '_create_udf',
 '_create_window_function',
 '_functions',
 '_functions_1_4',
 '_functions_1_6',
 '_functions_2_1',
 '_functions_deprecated',
 '_lit_doc',
 '_message',
 '_string_functions',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_window_functions',
 '_wrap_deprecated_function',
 'abs',
 'acos',
 'add_months',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'asc',
 'ascii',
 'asin',
 'atan',
 'atan2',
 'avg',
 'base64',
 'bin',
 'bitwiseNOT',
 'blacklist',
 'broadcast',
 'bround',
 'cbrt',
 'ceil',
 'coalesce',
 'col',
 'collect_lis